In [ ]:
!pip install transformers torch gradio PyPDF2



import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import PyPDF2
import io

# Load model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Text generation function
def generate_response(prompt, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True)
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

# PDF extraction
def extract_text_pdf(pdf_file):
    if pdf_file is None:
        return ""
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file.read()))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

# Requirement analysis
def requirement_analysis(pdf_file, prompt_text):
    if pdf_file is not None:
        content = extract_text_pdf(pdf_file)
        analysis_prompt = (
            "Please analyze the following document and extract key software requirements. "
            "Organize them into functional and non-functional requirements:\n\n" + content
        )
    else:
        analysis_prompt = (
            "Analyze the following requirements and organize them into functional and non-functional requirements:\n\n"
            + prompt_text
        )
    return generate_response(analysis_prompt, max_length=1200)

# Code generation
def code_generation(prompt, language):
    code_prompt = (
        f"Generate {language} code based on the following requirements:\n\n{prompt}\n\nCode:"
    )
    return generate_response(code_prompt, max_length=1200)

# Bug fixer
def bugfixer(code_snippet, language):
    fix_prompt = (
        f"Analyze the following {language} code and fix any bugs or issues. "
        f"Provide the corrected version:\n\n{code_snippet}\n\nFixed Code:"
    )
    return generate_response(fix_prompt, max_length=1200)

# Summarizer
def summarizer(pdf_file, prompt_text):
    if pdf_file is not None:
        content = extract_text_pdf(pdf_file)
        summary_prompt = (
            "Summarize the following document in a concise and clear manner:\n\n" + content
        )
    else:
        summary_prompt = (
            "Summarize the following text in a concise and clear manner:\n\n" + prompt_text
        )
    return generate_response(summary_prompt, max_length=800)

# Chatbot logic
def chat(user_input, history):
    history = history or []
    response = generate_response(user_input)
    history.append((user_input, response))
    return history, response

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 🧠 AI Code Analyzer, Generator & Chatbot")

    with gr.Tabs():
        # Tab 1: Requirement Analysis
        with gr.TabItem("Requirement Analysis"):
            with gr.Row():
                with gr.Column():
                    pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"])
                    prompt_input = gr.Textbox(
                        label="Write requirements here",
                        placeholder="Describe your requirements...",
                        lines=5
                    )
                    analyze_btn = gr.Button("Analyze")
                with gr.Column():
                    analysis_output = gr.Textbox(label="Requirements Analysis", lines=20)
            analyze_btn.click(requirement_analysis, inputs=[pdf_upload, prompt_input], outputs=analysis_output)

        # Tab 2: Code Generation
        with gr.TabItem("Code Generation"):
            with gr.Row():
                with gr.Column():
                    code_prompt = gr.Textbox(
                        label="Code Requirements",
                        placeholder="Describe your code requirements...",
                        lines=5
                    )
                    language_dropdown = gr.Dropdown(
                        choices=["Python", "Java", "JavaScript", "C++", "C#", "PHP", "Go", "Rust"],
                        label="Programming Language",
                        value="Python"
                    )
                    generate_btn = gr.Button("Generate Code")
                with gr.Column():
                    code_output = gr.Textbox(label="Generated Code", lines=20)
            generate_btn.click(code_generation, inputs=[code_prompt, language_dropdown], outputs=code_output)

        # Tab 3: Bug Fixer
        with gr.TabItem("Bug Fixer"):
            with gr.Row():
                with gr.Column():
                    buggy_code_input = gr.Textbox(
                        label="Paste Buggy Code",
                        placeholder="Insert your code with potential bugs...",
                        lines=10
                    )
                    bugfix_language_dropdown = gr.Dropdown(
                        choices=["Python", "Java", "JavaScript", "C++", "C#", "PHP", "Go", "Rust"],
                        label="Programming Language",
                        value="Python"
                    )
                    fix_btn = gr.Button("Fix Bugs")
                with gr.Column():
                    fixed_code_output = gr.Textbox(label="Fixed Code", lines=20)
            fix_btn.click(bugfixer, inputs=[buggy_code_input, bugfix_language_dropdown], outputs=fixed_code_output)

        # Tab 4: Summarizer
        with gr.TabItem("Summarizer"):
            with gr.Row():
                with gr.Column():
                    summary_pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"])
                    summary_prompt_input = gr.Textbox(
                        label="Or paste text to summarize",
                        placeholder="Enter text here...",
                        lines=5
                    )
                    summarize_btn = gr.Button("Summarize")
                with gr.Column():
                    summary_output = gr.Textbox(label="Summary", lines=20)
            summarize_btn.click(summarizer, inputs=[summary_pdf_upload, summary_prompt_input], outputs=summary_output)

        # Tab 5: Chatbot
        with gr.TabItem("Chatbot"):
            with gr.Row():
                with gr.Column():
                    chat_history = gr.Chatbot(label="Chat History")
                    user_message = gr.Textbox(
                        label="Your Message",
                        placeholder="Say something...",
                        lines=2
                    )
                    send_btn = gr.Button("Send")
                with gr.Column():
                    bot_response = gr.Textbox(label="Bot Response", lines=10)
            send_btn.click(chat, inputs=[user_message, chat_history], outputs=[chat_history, bot_response])

app.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/tmp/ipython-input-2650402687.py:182: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_history = gr.Chatbot(label="Chat History")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4bd18ea2035c3c41a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
